In [8]:
import requests
import pandas as pd
import time
from config import api_key, census_api_key
from census import Census

# Define the date range for the data query
begin_date = "01-2000"
end_date = "12-2023"
time_frame = f'?from={begin_date}&to={end_date}'

# List of crime types for which data will be fetched
crime = ["homicide", "rape", "robbery", "aggravated-assault", "arson", "burglary", "larceny", "motor-vehicle-theft"]

# Dictionary of states with their abbreviations and full names
states = {
    "AK": "Alaska", "AL": "Alabama", "AR": "Arkansas", "AZ": "Arizona",
    "CA": "California", "CO": "Colorado", "CT": "Connecticut", "DC": "District of Columbia",
    "DE": "Delaware", "FL": "Florida", "GA": "Georgia", "HI": "Hawaii",
    "IA": "Iowa", "ID": "Idaho", "IL": "Illinois", "IN": "Indiana",
    "KS": "Kansas", "KY": "Kentucky", "LA": "Louisiana", "MA": "Massachusetts",
    "MD": "Maryland", "ME": "Maine", "MI": "Michigan", "MN": "Minnesota",
    "MO": "Missouri", "MS": "Mississippi", "MT": "Montana", "NC": "North Carolina",
    "ND": "North Dakota", "NE": "Nebraska", "NH": "New Hampshire", "NJ": "New Jersey",
    "NM": "New Mexico", "NV": "Nevada", "NY": "New York", "OH": "Ohio",
    "OK": "Oklahoma", "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island",
    "SC": "South Carolina", "SD": "South Dakota", "TN": "Tennessee", "TX": "Texas",
    "UT": "Utah", "VA": "Virginia", "VT": "Vermont", "WA": "Washington",
    "WI": "Wisconsin", "WV": "West Virginia", "WY": "Wyoming"
}

# Function to fetch crime data from the API
def fetch_crime_data(crime, state_code, time_frame, api_key):
    base_url = f'https://api.usa.gov/crime/fbi/cde/summarized/state/{state_code}/{crime}{time_frame}{api_key}'
    response = requests.get(base_url)
    if response.status_code != 200:
        raise ValueError(f"API call failed with status {response.status_code}")
    return response.json()

# Function to process the fetched crime data into a DataFrame
def process_crime_data(crime_data, state_name):
    # Define the columns to extract from the crime data
    data_columns = {
        "Total Crimes": f"offenses.actuals.{state_name}",
        "Clearances": f"offenses.actuals.{state_name} Clearances",
        "Crime(Per 100k)": f"offenses.rates.{state_name}",
        "Prosecutions(Per 100k)": f"offenses.rates.{state_name} Clearances",
        "Total Pop": f"populations.population.{state_name}",
        "Partic Pop": f"populations.participated_population.{state_name}",
        "Pop Coverage": f"tooltips.Percent of Population Coverage.{state_name}"
    }
    
    result = {}
    for col_name, key_path in data_columns.items():
        # Navigate through the nested JSON data structure to extract values
        keys = key_path.split(".")
        data = crime_data
        for key in keys:
            data = data.get(key, {})
        result[col_name] = list(data.values())
    
    return pd.DataFrame(result, index=list(data.keys()))

# Create a dictionary to hold data for each crime type
crime_data_dict = {c: [] for c in crime}

# Loop over each state and each crime type to fetch and process the data
for state_code, state_name in states.items():
    for c in crime:
        try:
            # Fetch crime data for the state and crime
            crime_data = fetch_crime_data(c, state_code, time_frame, api_key)
            
            # Process the crime data into a DataFrame
            df_crime_data = process_crime_data(crime_data, state_name)
            
            # Add state abbreviation and crime type as columns
            df_crime_data["State"] = state_code
            df_crime_data["Crime"] = c
            
            # Convert the index (months) to DateTime format and reset the index
            df_crime_data.index = pd.to_datetime(df_crime_data.index, format='%m-%Y', errors='coerce')
            df_crime_data = df_crime_data.reset_index().rename(columns={"index": "Month"})
            
            # Append the data to the crime_data_dict under the corresponding crime type
            crime_data_dict[c].append(df_crime_data)
            
            print(f"Processed data for {state_name} ({state_code}) - {c}")
        except Exception as e:
            print(f"Error processing {state_name} ({state_code}) - {c}: {e}")
    
    # Pause to avoid overloading the API
    time.sleep(2)

# Define violent and property crime categories
violent_crimes = ["homicide", "rape", "robbery", "aggravated-assault"]
property_crimes = ["arson", "burglary", "larceny", "motor-vehicle-theft"]

# Initialize an empty DataFrame to store the final results
final_data = pd.DataFrame()

for c, data in crime_data_dict.items():
    # Combine all crime data for all crime types into a single DataFrame
    combined_data = pd.concat([pd.concat(data, ignore_index=True) for data in crime_data_dict.values()], ignore_index=True)

    # Ensure Month is in DateTime format and add a Year column
    combined_data["Month"] = pd.to_datetime(combined_data["Month"], errors="coerce")
    combined_data["Year"] = combined_data["Month"].dt.year

    # Add a Crime Category column (Violent or Property)
    combined_data["Crime Category"] = combined_data["Crime"].apply(
        lambda x: "Violent" if x in violent_crimes else "Property"
    )

    # Group data by State, Year, and Crime Category, then aggregate
    grouped = (
        combined_data.groupby(["State", "Year", "Crime Category"])
        .agg(
            {
                "Total Crimes": "sum",
                "Clearances": "sum",
                "Crime(Per 100k)": "mean",  # Use mean for per 100k rates
                "Prosecutions(Per 100k)": "mean",
                "Total Pop": "last",  # Use the last population data for each group
                "Partic Pop": "last",
                "Pop Coverage": "last",
            }
        )
        .reset_index()
    )

    # Pivot data to separate Violent and Property Crimes into columns
    pivoted_data = grouped.pivot(
        index=["State", "Year"], columns="Crime Category", values=["Total Crimes", "Clearances", "Crime(Per 100k)"]
    )

    # Flatten MultiIndex columns
    pivoted_data.columns = ["_".join(col).strip() for col in pivoted_data.columns]
    pivoted_data.reset_index(inplace=True)

    # Merge back population and coverage data (as they are not crime-category specific)
    extra_columns = grouped[["State", "Year", "Total Pop", "Partic Pop", "Pop Coverage"]].drop_duplicates()
    final_data = pd.merge(pivoted_data, extra_columns, on=["State", "Year"], how="left")

# Rename columns for clarity
final_data.rename(
    columns={
        "Total Crimes_Violent": "Violent_Total_Crimes",
        "Total Crimes_Property": "Property_Total_Crimes",
        "Clearances_Violent": "Violent_Clearances",
        "Clearances_Property": "Property_Clearances",
        "Crime(Per 100k)_Violent": "Violent_Crime_Rate",
        "Crime(Per 100k)_Property": "Property_Crime_Rate",
    },
    inplace=True,
)

# Calculate Derived Metrics
final_data["Total Crimes"] = (
    final_data["Violent_Total_Crimes"] + final_data["Property_Total_Crimes"]
)
final_data["Total Clearances"] = (
    final_data["Violent_Clearances"] + final_data["Property_Clearances"]
)
final_data["Prosecuted %"] = (
    (final_data["Total Clearances"] / final_data["Total Crimes"]) * 100
).round(2)
final_data["Total_Crime_Rate"] = (
    ((final_data["Violent_Total_Crimes"] + final_data["Property_Total_Crimes"]) / final_data["Total Pop"] * 100000).round(2)
)

# Round the crime rates to 2 decimal places
final_data["Violent_Crime_Rate"] = final_data["Violent_Crime_Rate"].round(2)
final_data["Property_Crime_Rate"] = final_data["Property_Crime_Rate"].round(2)

# Reorganize columns
final_columns = [
    "State",
    "Year",
    "Total Crimes",
    "Violent_Total_Crimes",
    "Property_Total_Crimes",
    "Total Clearances",
    "Prosecuted %",
    "Violent_Crime_Rate",
    "Property_Crime_Rate",
    "Total_Crime_Rate",
    "Total Pop",
    "Partic Pop",
    "Pop Coverage",
]
final_data = final_data[final_columns]

# State Code flip for Census API
state_fips = {
    "AL": "01", "AK": "02", "AZ": "04", "AR": "05", "CA": "06", "CO": "08", "CT": "09", "DE": "10", "DC": "11",
    "FL": "12", "GA": "13", "HI": "15", "ID": "16", "IL": "17", "IN": "18", "IA": "19", "KS": "20", "KY": "21",
    "LA": "22", "ME": "23", "MD": "24", "MA": "25", "MI": "26", "MN": "27", "MS": "28", "MO": "29", "MT": "30",
    "NE": "31", "NV": "32", "NH": "33", "NJ": "34", "NM": "35", "NY": "36", "NC": "37", "ND": "38", "OH": "39",
    "OK": "40", "OR": "41", "PA": "42", "RI": "44", "SC": "45", "SD": "46", "TN": "47", "TX": "48", "UT": "49",
    "VT": "50", "VA": "51", "WA": "53", "WV": "54", "WI": "55", "WY": "56"
}



# Function to fetch poverty count for a state in a specific year
def get_poverty_count(state_code, year):
    # Initialize Census API
    c = Census(census_api_key, year=year)

    # Census API query to get poverty count (B17001_002E)
    try:
        census_data = c.acs5.get(
            ("B17001_002E"),  # Poverty count variable
            {'for': f'state:{state_code}'}  # State FIPS code and year
        )
        # Return the poverty count
        return census_data[0]['B17001_002E']
    except Exception as e:
        print(f"Poverty data doesn't exist for {state_code} for {year}.")
        return None  # Return None if data fetch fails

# Create a new column for the Poverty Rate
final_data['Poverty Count'] = None
final_data['Poverty Rate'] = None

# Loop through each row in the final_data DataFrame to fetch poverty data
for index, row in final_data.iterrows():
    state = row['State']
    state_code = state_fips[f"{state}"]
    year = row['Year']
    if year > 2008:
        # Fetch poverty count for the state and year
        poverty_count = get_poverty_count(state_code, year)
    
        if poverty_count is not None:
            final_data.at[index, 'Poverty Count'] = poverty_count
        
            # Calculate the poverty rate: Poverty Rate = (Poverty Count / Total Population) * 100
            total_pop = row['Total Pop']
            poverty_rate = (poverty_count / total_pop) * 100 if total_pop > 0 else 0
            final_data.at[index, 'Poverty Rate'] = round(poverty_rate, 2)  # Round to 2 decimal places
            print(f"Processed poverty data for {state} for {year}")

# Save the updated final data to a CSV file
final_data.to_csv("../Resources/State/State_Crime_Data_With_Poverty.csv", index=False)

print("State-level aggregated crime data with poverty data saved.")

# Save the final aggregated data to a CSV file
final_data.to_csv("../Resources/State/State_Crime_Poverty.csv", index=False)

print("State-level aggregated crime data saved to '../Resources/State/State_Crime_Poverty.csv'")



Processed data for Alaska (AK) - homicide
Processed data for Alaska (AK) - rape
Processed data for Alaska (AK) - robbery
Processed data for Alaska (AK) - aggravated-assault
Processed data for Alaska (AK) - arson
Processed data for Alaska (AK) - burglary
Processed data for Alaska (AK) - larceny
Processed data for Alaska (AK) - motor-vehicle-theft
Processed data for Alabama (AL) - homicide
Processed data for Alabama (AL) - rape
Processed data for Alabama (AL) - robbery
Processed data for Alabama (AL) - aggravated-assault
Processed data for Alabama (AL) - arson
Processed data for Alabama (AL) - burglary
Processed data for Alabama (AL) - larceny
Processed data for Alabama (AL) - motor-vehicle-theft
Processed data for Arkansas (AR) - homicide
Processed data for Arkansas (AR) - rape
Processed data for Arkansas (AR) - robbery
Processed data for Arkansas (AR) - aggravated-assault
Processed data for Arkansas (AR) - arson
Processed data for Arkansas (AR) - burglary
Processed data for Arkansas (A